In [10]:
import h5py
import tensorflow as tf

print(h5py.__version__)
print(tf.__version__)


3.12.1
2.19.1


Import coding and config

In [11]:
import numpy as np
import cv2
import math
import tensorflow as tf
from tensorflow import keras

print("TF:", tf.__version__)
print("GPU devices:", tf.config.list_physical_devices("GPU"))
print("OpenCV:", cv2.__version__)

# ===== PATHS =====
MODEL_PATH = "lane_model.h5" #model
CALIB_PATH = "stereo_params_debug.npz" #hasil kalibrasi stereo camera

# ===== CAMERA INDEX =====
CAM_L = 2
CAM_R = 1

# ===== MASK / GEOMETRY =====
MASK_THRESH = 0.5
ROI_Y_START_RATIO = 0.55
NUM_SLICES = 30

# ===== STEERING -> VOLTAGE =====
MAX_STEER = 30.0
V_CENTER = 2.5
V_MIN = 1.3
V_MAX = 3.7


TF: 2.19.1
GPU devices: []
OpenCV: 4.10.0


Cell 2 — Load Stereo Calibration (.npz) + Print Keys

In [12]:
calib = np.load(CALIB_PATH)
print("NPZ keys:", calib.files)

# Print shapes for sanity
for k in calib.files:
    v = calib[k]
    if isinstance(v, np.ndarray):
        print(k, v.shape, v.dtype)


NPZ keys: ['keep_idx', 'mean_errs', 'K1', 'dist1', 'K2', 'dist2', 'R', 'T']
keep_idx (22,) int64
mean_errs (25,) float32
K1 (3, 3) float64
dist1 (1, 5) float64
K2 (3, 3) float64
dist2 (1, 5) float64
R (3, 3) float64
T (3, 1) float64


In [13]:
# Extract baseline distance between stereo cameras
if "T" in calib.files:
    T = calib["T"]
    print("\nTranslation vector (T):", T)
    print("T shape:", T.shape)
    
    # The baseline distance is typically the magnitude of T or T[0] for horizontal stereo
    if T.size == 3:
        baseline = np.linalg.norm(T)
        print(f"\nStereo Baseline Distance: {baseline:.4f} cm")
        print(f"  Tx (horizontal): {T[0,0]:.4f} cm")
        print(f"  Ty (vertical): {T[1,0]:.4f} cm")
        print(f"  Tz (depth): {T[2,0]:.4f} cm")
    elif T.size == 1:
        print(f"\nStereo Baseline Distance: {T[0,0]:.4f} cm")
else:
    print("\n[WARNING] Translation vector 'T' not found in calibration file")



Translation vector (T): [[ 0.22665312]
 [ 0.010976  ]
 [-0.05325579]]
T shape: (3, 1)

Stereo Baseline Distance: 0.2331 cm
  Tx (horizontal): 0.2267 cm
  Ty (vertical): 0.0110 cm
  Tz (depth): -0.0533 cm


Cell 3 — Get Calibration Matrices + Build Rectify Maps

In [14]:
import numpy as np
import cv2

CALIB_PATH = "stereo_calibration.npz"   # ganti sesuai nama file kamu
calib = np.load(CALIB_PATH)

print("[INFO] Keys in npz:", calib.files)

def pick(calib, candidates, default=None, required=False):
    for c in candidates:
        if c in calib.files:
            return calib[c]
    if required:
        raise KeyError(f"Missing keys. Tried: {candidates}")
    return default

# ---- Intrinsics & Distortion (wajib) ----
K1 = pick(calib, ["K1","M1","cameraMatrix1","mtxL","mtx1"], required=True)
D1 = pick(calib, ["D1","dist1","distCoeffs1","distL","d1"], required=True)
K2 = pick(calib, ["K2","M2","cameraMatrix2","mtxR","mtx2"], required=True)
D2 = pick(calib, ["D2","dist2","distCoeffs2","distR","d2"], required=True)

# ---- Image size (wajib untuk mapping) ----
# Coba ambil dari npz, kalau ga ada kamu set manual sesuai resolusi webcam
if "imageSize" in calib.files:
    sz = calib["imageSize"].astype(int).flatten()
    if len(sz) >= 2:
        W, H = int(sz[0]), int(sz[1])
    else:
        W, H = 640, 480
else:
    W, H = 640, 480  # <-- kalau resolusi kamu beda, ganti ini

img_shape = (W, H)

# ---- Case A: map sudah ada -> pakai langsung ----
map1x = pick(calib, ["map1x"])
map1y = pick(calib, ["map1y"])
map2x = pick(calib, ["map2x"])
map2y = pick(calib, ["map2y"])

if map1x is not None and map1y is not None and map2x is not None and map2y is not None:
    print("[OK] Found rectify maps inside NPZ. Using saved maps.")
else:
    print("[INFO] Rectify maps NOT found. Computing stereoRectify -> maps...")

    # ---- Need extrinsics R & T to compute rectify ----
    R = pick(calib, ["R"], required=True)
    T = pick(calib, ["T"], required=True)

    # ---- Compute stereoRectify outputs ----
    R1, R2, P1, P2, Q, _, _ = cv2.stereoRectify(
        K1, D1, K2, D2, img_shape, R, T,
        flags=cv2.CALIB_ZERO_DISPARITY, alpha=0
    )

    # ---- Build rectify maps ----
    map1x, map1y = cv2.initUndistortRectifyMap(K1, D1, R1, P1, img_shape, cv2.CV_32FC1)
    map2x, map2y = cv2.initUndistortRectifyMap(K2, D2, R2, P2, img_shape, cv2.CV_32FC1)

    print("[OK] stereoRectify + maps created.")

print("[OK] Rectification maps ready:", img_shape)


[INFO] Keys in npz: ['K1', 'dist1', 'K2', 'dist2', 'R', 'T', 'image_size']
[INFO] Rectify maps NOT found. Computing stereoRectify -> maps...
[OK] stereoRectify + maps created.
[OK] Rectification maps ready: (640, 480)


Cell 4 — Load Lane Model + Mask Function

In [15]:
print("[INFO] Loading lane model:", MODEL_PATH)
model = keras.models.load_model(MODEL_PATH)
print("[OK] Loaded. input_shape:", model.input_shape)

_, IN_H, IN_W, IN_C = model.input_shape
if IN_C != 3:
    raise ValueError("Model input channel != 3. Check model.input_shape")

def predict_lane_mask(frame_bgr, thresh=MASK_THRESH):
    rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    inp = cv2.resize(rgb, (IN_W, IN_H)).astype(np.float32) / 255.0
    inp = np.expand_dims(inp, axis=0)

    pred = model.predict(inp, verbose=0)[0]  # (IN_H, IN_W, 1)

    mask = (pred[..., 0] > thresh).astype(np.uint8)
    mask_full = cv2.resize(mask, (W, H), interpolation=cv2.INTER_NEAREST)
    return mask_full


[INFO] Loading lane model: lane_model.h5


[OK] Loaded. input_shape: (None, 256, 256, 3)


Cell 5 — Boundary (Curved) + Angle + Steering + Voltage

In [16]:
def extract_boundaries_and_angles_curved(mask_full, roi_y_start_ratio=0.55, num_slices=30):
    h, w = mask_full.shape
    y_start = int(h * roi_y_start_ratio)
    ys_scan = np.linspace(y_start, h-1, num_slices).astype(int)

    left_points, right_points = [], []
    for y in ys_scan:
        xs = np.where(mask_full[y, :] == 1)[0]
        if len(xs) < 5:
            continue
        left_points.append((xs[0], y))
        right_points.append((xs[-1], y))

    def fit_curve_and_angle(points, n_samples=80):
        if len(points) < 6:
            return None, []
        pts = np.array(points)
        xs, ys = pts[:,0], pts[:,1]

        try:
            a2, a1, a0 = np.polyfit(ys, xs, 2)   # x = a2*y^2 + a1*y + a0
        except np.linalg.LinAlgError:
            a1, a0 = np.polyfit(ys, xs, 1)
            a2 = 0.0

        y_min, y_max = ys.min(), ys.max()
        curve_ys = np.linspace(y_min, y_max, n_samples)
        curve_xs = a2*curve_ys**2 + a1*curve_ys + a0

        curve_pts = [(int(np.clip(x,0,w-1)), int(np.clip(y,0,h-1))) for x,y in zip(curve_xs, curve_ys)]

        dx_dy = 2*a2*y_max + a1
        angle_deg = math.degrees(math.atan(dx_dy))
        return angle_deg, curve_pts

    left_angle, left_curve = fit_curve_and_angle(left_points)
    right_angle, right_curve = fit_curve_and_angle(right_points)

    return left_angle, right_angle, left_curve, right_curve

def compute_steering_angle(left_angle, right_angle, w_left=0.5, w_right=0.5):
    if left_angle is None or right_angle is None:
        return None
    s = w_left + w_right
    wL, wR = w_left/s, w_right/s
    return wL*left_angle + wR*right_angle

def steering_to_voltage(steering_angle):
    if steering_angle is None:
        return None
    a = float(np.clip(steering_angle, -MAX_STEER, MAX_STEER))
    scale = (V_MAX - V_MIN) / (2*MAX_STEER)
    return V_CENTER + a*scale


Cell 6 — Kalman Filter MATRIX 2D (Angle + Angular Rate)

In [17]:
class KalmanMatrix2D:
    """
    x = [theta; theta_rate]
    """
    def __init__(self, dt=1/30, q_theta=0.05, q_rate=1.0, r_meas=9.0, init_P=10.0):
        self.dt = float(dt)
        self.x = np.array([[0.0],[0.0]], dtype=np.float32)
        self.P = np.eye(2, dtype=np.float32) * float(init_P)

        self.A = np.array([[1.0, self.dt],
                           [0.0, 1.0]], dtype=np.float32)
        self.H = np.array([[1.0, 0.0]], dtype=np.float32)
        self.Q = np.array([[q_theta, 0.0],
                           [0.0, q_rate]], dtype=np.float32)
        self.R = np.array([[r_meas]], dtype=np.float32)
        self.I = np.eye(2, dtype=np.float32)

    def update(self, z):
        # Predict
        x_pred = self.A @ self.x
        P_pred = self.A @ self.P @ self.A.T + self.Q

        if z is None or (not np.isfinite(z)):
            self.x, self.P = x_pred, P_pred
            return float(self.x[0,0])

        # Update
        z = np.array([[float(z)]], dtype=np.float32)
        y = z - (self.H @ x_pred)                  # innovation
        S = self.H @ P_pred @ self.H.T + self.R    # innovation cov
        K = P_pred @ self.H.T @ np.linalg.inv(S)   # Kalman gain

        self.x = x_pred + K @ y
        self.P = (self.I - K @ self.H) @ P_pred
        return float(self.x[0,0])


Cell 7 — RUN Realtime Stereo + Lane + Angles + KF + Steering

In [18]:
capL = cv2.VideoCapture(CAM_L)
capR = cv2.VideoCapture(CAM_R)

for cap in (capL, capR):
    cap.set(cv2.CAP_PROP_FRAME_WIDTH,  W)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, H)

if not capL.isOpened() or not capR.isOpened():
    raise RuntimeError("Cannot open stereo cameras. Check CAM_L/CAM_R indices.")

kfL = KalmanMatrix2D(dt=1/30, q_theta=0.05, q_rate=1.0, r_meas=9.0)
kfR = KalmanMatrix2D(dt=1/30, q_theta=0.05, q_rate=1.0, r_meas=9.0)

print("[INFO] Running... Press 'q' to quit.")

while True:
    retL, frameL = capL.read()
    retR, frameR = capR.read()
    if not retL or not retR:
        print("[WARN] Failed to grab frames")
        break

    # 1) Rectify
    rectL = cv2.remap(frameL, map1x, map1y, cv2.INTER_LINEAR)
    rectR = cv2.remap(frameR, map2x, map2y, cv2.INTER_LINEAR)

    # 2) Lane segmentation (LEFT only)
    mask = predict_lane_mask(rectL, thresh=MASK_THRESH)

    # 3) Angles raw
    left_raw, right_raw, left_curve, right_curve = extract_boundaries_and_angles_curved(
        mask, roi_y_start_ratio=ROI_Y_START_RATIO, num_slices=NUM_SLICES
    )

    # 4) Kalman filter on angles
    left_kf  = kfL.update(left_raw)
    right_kf = kfR.update(right_raw)

    # 5) Steering + voltage
    steer = compute_steering_angle(left_kf, right_kf, 0.5, 0.5)
    volt  = steering_to_voltage(steer)

    # print raw vs kf
    print(f"L raw:{left_raw} -> KF:{left_kf:.2f} | R raw:{right_raw} -> KF:{right_kf:.2f} | Steer:{steer} | V:{volt}")

    # ===== VISUAL =====
    vis = rectL.copy()

    # overlay mask
    mask_color = np.zeros_like(vis, dtype=np.uint8)
    mask_color[mask == 1] = (0, 255, 0)
    vis = cv2.addWeighted(vis, 1.0, mask_color, 0.35, 0)

    # draw curve points
    for (x,y) in left_curve:
        cv2.circle(vis, (x,y), 2, (255,0,0), -1)
    for (x,y) in right_curve:
        cv2.circle(vis, (x,y), 2, (0,0,255), -1)

    # text output (before/after KF)
    cv2.putText(vis, f"L raw:{left_raw if left_raw is not None else 'NA'} | KF:{left_kf:.2f}",
                (20,40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    cv2.putText(vis, f"R raw:{right_raw if right_raw is not None else 'NA'} | KF:{right_kf:.2f}",
                (20,75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    cv2.putText(vis, f"Steer:{steer if steer is not None else 'NA'} deg",
                (20,110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    cv2.putText(vis, f"Volt:{volt if volt is not None else 'NA'} V",
                (20,145), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

    # show right rectified for debug
    show = cv2.hconcat([vis, rectR])
    cv2.imshow("Stereo (Left processed | Right rectified)", show)

    if (cv2.waitKey(1) & 0xFF) == ord('q'):
        break

capL.release(); capR.release()
cv2.destroyAllWindows()


[INFO] Running... Press 'q' to quit.
L raw:87.09267294557719 -> KF:45.97 | R raw:-43.74844204143512 -> KF:-23.09 | Steer:11.439290046691895 | V:2.957571601867676
L raw:48.30911046532929 -> KF:46.82 | R raw:-9.899150013510042 -> KF:-18.51 | Steer:14.157132148742676 | V:3.066285285949707
L raw:58.80203210991368 -> KF:50.02 | R raw:20.403315462104967 -> KF:-8.24 | Steer:20.888317108154297 | V:3.335532684326172
L raw:-74.46067438208362 -> KF:23.23 | R raw:-75.8816701811362 -> KF:-22.78 | Steer:0.22316741943359375 | V:2.5089266967773436
L raw:-39.24321723408464 -> KF:11.38 | R raw:-41.228362939915336 -> KF:-26.31 | Steer:-7.468360900878906 | V:2.2012655639648435
L raw:73.96271280998359 -> KF:21.43 | R raw:-81.78500940236994 -> KF:-35.75 | Steer:-7.162836074829102 | V:2.213486557006836
L raw:82.76049876789898 -> KF:30.77 | R raw:-84.98543066565405 -> KF:-43.77 | Steer:-6.5013885498046875 | V:2.2399444580078125
L raw:85.49934934770448 -> KF:38.98 | R raw:-11.932241250974917 -> KF:-39.66 | Ste